In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras import regularizers
from keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
import os

In [ ]:
image_path = '/kaggle/input/utkface-filtered-images'

**Завантаження зображень з папки, попередня обробка та формування набору даних (X — зображення, Y — мітки віку)**

In [ ]:
width = 100
height = 100

X = []
Y = []

# Проходимо по всіх файлах в папці
for folder_name, _, filenames in os.walk(image_path):
    for file in filenames:
        # Створюємо повний шлях до файлу
        file_path = os.path.join(folder_name, file)
        
        # Отримуємо мітку віку з назви файлу (перша частина до символу '_')
        age = int(file.split('_')[0]) 
        
        # Відкриваємо зображення
        image = Image.open(file_path)
        image = image.convert('RGB')
        image = image.resize((width, height))
        
        # Додаємо зображення і мітку до відповідних списків
        X.append(np.array(image))
        Y.append(age)

X = np.array(X)
Y = np.array(Y)

print(f"Зображень: {X.shape[0]}, Міток: {Y.shape[0]}")

**Нормалізація пікселів зображень до діапазону [0, 1]**

In [ ]:
X = X.astype('float32')
X /= 255.0

print('Min: %.3f, Max: %.3f' % (X.min(), X.max()))

**Розбиття даних на тренувальний, валідаційний і тестовий набори з конверсією міток у float32**

In [ ]:
seed = 42
# Спочатку 70% train, 30% temp
X_train, X_temp, Y_train, Y_temp = train_test_split(
    X, Y,
    test_size=0.3,
    random_state=seed,
    stratify=Y,
    shuffle=True
)

# Потім temp (30%) ділимо навпіл: 15% val, 15% test
X_val, X_test, Y_val, Y_test = train_test_split(
    X_temp, Y_temp,
    test_size=0.5,
    random_state=seed,
    stratify=Y_temp,
    shuffle=True
)

# Приводимо до потрібного типу (якщо треба для моделі)
Y_train = np.array(Y_train, dtype=np.float32)
Y_val = np.array(Y_val, dtype=np.float32)
Y_test = np.array(Y_test, dtype=np.float32)

print("Y_train shape:", Y_train.shape)
print("Y_val shape:", Y_val.shape)
print("Y_test shape:", Y_test.shape)


**Налаштування генераторів зображень: аугментація для тренувальних, без змін для тестових даних**

In [ ]:
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
)

test_datagen = ImageDataGenerator()

**Налаштування callback-функцій та гіперпараметрів навчання**

In [ ]:
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from timeit import default_timer as timer

# Callback для вимірювання часу тренування кожної епохи
class TimingCallback(Callback):
    def __init__(self):
        self.logs = []

    def on_epoch_begin(self, epoch, logs=None):
        self.starttime = timer()

    def on_epoch_end(self, epoch, logs=None):
        self.logs.append(timer() - self.starttime)

# Callback для ранньої зупинки навчання
early_stopping = EarlyStopping(
    patience=6,  # Чекаємо 6 епох перед зупинкою
    min_delta=0.01,  # Якщо зміни у втраті менші за 1%, зупиняємось
    verbose=1,
    mode='min',
    monitor='val_loss'
)

# Callback для зменшення learning rate при плато
reduce_learning_rate = ReduceLROnPlateau(
    monitor="val_loss",
    patience=5,  # Якщо val_loss не покращується 5 епох, зменшуємо lr
    factor=0.1,  # Зменшуємо learning rate у 10 разів
    cooldown=4,  # Чекаємо 4 епохи перед повторним зменшенням
    min_lr=1e-6,  # Мінімальне значення learning rate
    verbose=1
)

# Callback для вимірювання часу епох
time_callback = TimingCallback()

# Гіперпараметри
lr = 0.1
epochs = 30 
batch_size = 32
results = {}

**Кодування вікових значень у категорії та one-hot представлення**

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical

Y_train_encoded = Y_train.flatten()
Y_val_encoded = Y_val.flatten()
Y_test_encoded = Y_test.flatten()

bins = [0, 9, 20, 26, 34, 45, 60, 100] 

Y_train_encoded = np.digitize(Y_train_encoded, bins, right=True) - 1
Y_val_encoded = np.digitize(Y_val_encoded, bins, right=True) - 1
Y_test_encoded = np.digitize(Y_test_encoded, bins, right=True) - 1

if np.max(Y_train_encoded) >= 7 or np.max(Y_val_encoded) >= 7 or np.max(Y_test_encoded) >= 7:
    print("Warning: There are values greater than or equal to 7!")

# Перетворення на one-hot encoding
Y_train_encoded = to_categorical(Y_train_encoded, num_classes=7)
Y_val_encoded = to_categorical(Y_val_encoded, num_classes=7)
Y_test_encoded = to_categorical(Y_test_encoded, num_classes=7)

print("After one-hot encoding - Y_train_encoded shape:", Y_train_encoded.shape)
print("After one-hot encoding - Y_val_encoded shape:", Y_val_encoded.shape)
print("After one-hot encoding - Y_test_encoded shape:", Y_test_encoded.shape)


**Генератор навчальних даних з аугментацією для регресійної моделі**

In [ ]:
train_generator = train_datagen.flow(
    X_train, Y_train,
    batch_size=batch_size
)

**Побудова та навчання регресійної моделі на основі MobileNet**

In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (BatchNormalization, GlobalAveragePooling2D, 
                                     Dropout, Dense)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.utils.class_weight import compute_class_weight
import pickle

input_shape = (100, 100)
base_mobilenet_model = MobileNet(
    input_shape=(input_shape[0], input_shape[1], 3),
    include_top=False,
    weights=None
)

mobilenet_model_reg = Sequential([
    BatchNormalization(input_shape=(input_shape[0], input_shape[1], 3)),
    base_mobilenet_model,
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(1, activation='linear')
])

mobilenet_model_reg.compile(
    optimizer=Adam(),
    loss='mean_squared_error', 
    metrics=['mae']
)

# Навчання MobileNet
mobilenet_model_reg_history = mobilenet_model_reg.fit(
    train_generator,
    batch_size=batch_size,
    epochs=30,
    validation_data=(X_val, Y_val),
    callbacks=[reduce_learning_rate, time_callback],
    verbose=True
)

mobilenet_model_reg.save("mobilenet_age_regression.h5")

with open("mobilenet_training_reg_history.pkl", "wb") as f:
    pickle.dump(mobilenet_model_reg_history.history, f)

print("Модель і історія навчання збережені.")

**Створення і тренування власної згорткової нейронної мережі для задачі регресії**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, GlobalAveragePooling2D, Dense
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import pickle

input_shape = (100, 100, 3)

model = Sequential([
    Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=input_shape,
           kernel_regularizer=regularizers.l2(1e-5)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation="relu", padding="same",
           kernel_regularizer=regularizers.l2(1e-5)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation="relu", padding="same",
           kernel_regularizer=regularizers.l2(1e-5)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation="linear")
])

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="mean_squared_error",
    metrics=['mean_absolute_error']
)

model_history = model.fit(
    train_generator,
    validation_data=(X_val, Y_val),
    batch_size=batch_size,
    epochs=30,
    callbacks=[reduce_learning_rate, time_callback],
    verbose=True
)

model.save("custom_cnn_age_regression.h5")

with open("custom_cnn_training_history.pkl", "wb") as f:
    pickle.dump(model_history.history, f)

print("Модель і історія навчання збережені.")


**Створення та навчання регресійної моделі на базі VGG16**

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (BatchNormalization, GlobalAveragePooling2D, 
                                     Dropout, Dense)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.utils.class_weight import compute_class_weight
import pickle

input_shape = (100, 100)
base_vgg_model = VGG16(
    input_shape=(input_shape[0], input_shape[1], 3),
    include_top=False,
    weights=None
)

vgg_model_reg = Sequential([
    BatchNormalization(input_shape=(input_shape[0], input_shape[1], 3)),
    base_vgg_model,
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(1, activation='linear')
])

vgg_model_reg.compile(
    optimizer=Adam(),
    loss='mean_squared_error',
    metrics=['mae']
)

vgg_model_reg_history = vgg_model_reg.fit(
    train_generator,
    batch_size=batch_size,
    epochs=30,
    validation_data=(X_val, Y_val),
    callbacks=[reduce_learning_rate, time_callback],
    verbose=True
)

vgg_model_reg.save("vgg_age_regression.h5")

with open("vgg_model_reg_history.pkl", "wb") as f:
    pickle.dump(vgg_model_reg_history.history, f)

print("Модель і історія навчання збережені.")

**Генератор з аугментацією даних для класифікації**

In [ ]:
import tensorflow as tf

train_generator = train_datagen.flow(
    X_train,
    Y_train_encoded,
    batch_size=batch_size,
    shuffle=True
)

train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 7), dtype=tf.float32),
    )
).repeat()


import matplotlib.pyplot as plt
import numpy as np

augmented_images, augmented_labels = next(train_generator)

print("Shape:", augmented_images.shape)
print("Dtype:", augmented_images.dtype)
print("Min:", np.min(augmented_images))
print("Max:", np.max(augmented_images))

plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(augmented_images[i]) 
    plt.axis('off')
plt.tight_layout()
plt.show()


**Створення та навчання класифікаційної моделі на базі MobileNet**

In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (BatchNormalization, GlobalAveragePooling2D, 
                                     Dropout, Dense, Conv2D, MaxPooling2D, Flatten)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.utils.class_weight import compute_class_weight

input_shape = (100, 100)
base_mobilenet_model = MobileNet(
    input_shape=(input_shape[0], input_shape[1], 3),
    include_top=False,
    weights=None
)

mobilenet_model_clas = Sequential([
    BatchNormalization(input_shape=(input_shape[0], input_shape[1], 3)),
    base_mobilenet_model,
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(7, activation='softmax',kernel_regularizer=regularizers.l2(0.001)) 
])


mobilenet_model_clas.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

mobilenet_model_clas_history = mobilenet_model_clas.fit(
    train_dataset,
    steps_per_epoch=len(X_train) // batch_size,
    epochs=40,
    validation_data=(X_val, Y_val_encoded),
    callbacks=[reduce_learning_rate, time_callback],
    verbose=True
)

mobilenet_model_clas.save("mobilenet_age_clas.h5")

import pickle

with open("mobilenet_model_clas_history.pkl", "wb") as f:
    pickle.dump(mobilenet_model_clas_history.history, f)

print("Модель і історія навчання збережені.")

**Обчислення ваг класів для збалансування навчання класифікаційної моделі**

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

y_train_labels = np.argmax(Y_train_encoded, axis=1)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_labels),
    y=y_train_labels
)

class_weight_dict = dict(enumerate(class_weights))

print("Class weights:", class_weight_dict)

**Створення та навчання класифікаційної моделі на базі MobileNet із врахуванням ваг класів**

In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (BatchNormalization, GlobalAveragePooling2D, 
                                     Dropout, Dense, Conv2D, MaxPooling2D, Flatten)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.utils.class_weight import compute_class_weight

input_shape = (100, 100)
base_mobilenet_model = MobileNet(
    input_shape=(input_shape[0], input_shape[1], 3),
    include_top=False,
    weights=None
)

mobilenet_model_clas = Sequential([
    BatchNormalization(input_shape=(input_shape[0], input_shape[1], 3)),
    base_mobilenet_model,
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(7, activation='softmax',kernel_regularizer=regularizers.l2(0.001)) 
])

mobilenet_model_clas.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

mobilenet_model_clas_history = mobilenet_model_clas.fit(
    train_generator,
    epochs=40,
    validation_data=(X_val, Y_val_encoded),
    callbacks=[reduce_learning_rate, time_callback],
    verbose=True,
    class_weight=class_weight_dict  # ← працює!
)

mobilenet_model_clas.save("mobilenet_clas_regression.h5")

import pickle

with open("mobilenet_model_clas_history.pkl", "wb") as f:
    pickle.dump(mobilenet_model_clas_history.history, f)

print("Модель і історія навчання збережені.")

**Порівняння ефективності регресійних моделей за MAE**

In [ ]:
import pickle

with open('/kaggle/input/vgg-mobinet/vgg_training_history (1).pkl', 'rb') as f:
    vgg_history = pickle.load(f)

with open('/kaggle/input/vgg-mobinet/mobilenet_training_history (1).pkl', 'rb') as f:
    mobilenet_history = pickle.load(f)

with open('/kaggle/input/custom-simple-model-reg/custom_cnn_training_history.pkl', 'rb') as f:
    custom_history = pickle.load(f)
plt.figure(figsize=(12, 6))

plt.grid(True, linestyle='--', alpha=0.5)

plt.plot(vgg_history['mean_absolute_error'], label='vgg_history model_train')
plt.plot(vgg_history['val_mean_absolute_error'], '--', label='vgg_history model_val')

plt.plot(mobilenet_history['mae'], label='mobilenet_train')
plt.plot(mobilenet_history['val_mae'], '--', label='mobilenet_val')

plt.plot(custom_history['mean_absolute_error'], label='custom_train')
plt.plot(custom_history['val_mean_absolute_error'], '--', label='custom_val')

plt.title('Mean Absolute Error')
plt.legend()
plt.show()

**Підсумкові результати моделей за MAE**

In [ ]:
print("=== VGG Model ===")
print(f"Train MAE (last epoch): {vgg_history['mean_absolute_error'][-1]:.4f}")
print(f"Val MAE (last epoch):   {vgg_history['val_mean_absolute_error'][-1]:.4f}")

print("\n=== MobileNet Model ===")
print(f"Train MAE (last epoch): {mobilenet_history['mae'][-1]:.4f}")
print(f"Val MAE (last epoch):   {mobilenet_history['val_mae'][-1]:.4f}")

print("\n=== Custom CNN Model ===")
print(f"Train MAE (last epoch): {custom_history['mean_absolute_error'][-1]:.4f}")
print(f"Val MAE (last epoch):   {custom_history['val_mean_absolute_error'][-1]:.4f}")


**Візуалізація роботи регресійної моделі MobileNet на тестових даних**

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import load_model
import pickle

model_mobilenet_reg = load_model('/kaggle/input/vgg-mobinet/mobilenet_age_regression (1).h5')

with open('/kaggle/input/vgg-mobinet/mobilenet_training_history (1).pkl', 'rb') as f:
    history_mobilenet_reg = pickle.load(f)

predicted_ages = model_mobilenet_reg.predict(X_test).flatten()

true_ages = Y_test.flatten()

indices = random.sample(range(len(X_test)), 8)

plt.figure(figsize=(16, 8))

for i, idx in enumerate(indices):
    plt.subplot(2, 4, i + 1)
    plt.imshow(X_test[idx])
    plt.axis('off')
    plt.title(f"True: {int(true_ages[idx])}\nPred: {int(predicted_ages[idx])}",
              color='green' if abs(true_ages[idx] - predicted_ages[idx]) < 5 else 'red')

plt.tight_layout()
print(predicted_ages.shape)
plt.show()

**Візуалізація метрик навчання моделі MobileNet для задачі класифікації**

In [ ]:

model_mobilenet = load_model('/kaggle/input/mobilet-class-54-150/mobilenet_clas_regression (2).h5')

# Завантаження історії тренування (перевір точну назву файлу!)
with open('/kaggle/input/mobilet-class-54-150/mobilenet_model_clas_history.pkl', 'rb') as f:
    history_mobilenet_clas = pickle.load(f)
    
# Отримуємо дані для точності та втрат з історії тренування
train_accuracy = history_mobilenet_clas['accuracy']
val_accuracy = history_mobilenet_clas['val_accuracy']
train_loss = history_mobilenet_clas['loss']
val_loss = history_mobilenet_clas['val_loss']

plt.figure(figsize=(10, 10))

plt.subplot(2, 1, 1)
plt.plot(train_accuracy, label='Тренувальна точність')
plt.plot(val_accuracy, label='Валідаційна точність')
plt.title('Точність моделі')
plt.xlabel('Епохи')
plt.ylabel('Точність')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Втрати моделі')
plt.xlabel('Епохи')
plt.ylabel('Втрати')
plt.legend()

plt.tight_layout()
plt.show()

**Візуалізація роботи моделі класифікації MobileNet на тестових даних**

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import load_model
import pickle

model_mobilenet = load_model('/kaggle/input/mobilenet-class-56/mobilenet_clas_regression (3).h5')

with open('/kaggle/input/mobilenet-class-56/mobilenet_model_clas_history (1).pkl', 'rb') as f:
    history_mobilenet_clas = pickle.load(f)

predictions = model_mobilenet.predict(X_test)

true_labels = np.argmax(Y_test_encoded, axis=1)
predicted_labels = np.argmax(predictions, axis=1)

age_bins_labels = [
    '1–9 років', '10–20 років', '21–26 років', '27–34 років',
    '35–45 років', '45–60 років', '60–99 років'
]

indices = random.sample(range(len(X_test)), 8)

plt.figure(figsize=(16, 8))

for i, idx in enumerate(indices):
    plt.subplot(2, 4, i + 1)
    plt.imshow(X_test[idx])
    plt.axis('off')
    plt.title(f"True: {age_bins_labels[true_labels[idx]]}\nPred: {age_bins_labels[predicted_labels[idx]]}",
              color='green' if true_labels[idx] == predicted_labels[idx] else 'red')

plt.tight_layout()
print(predictions.shape)  # має бути (кількість_зразків, 7)
print("Графік зараз з'явиться")
plt.show()

**Оцінка якості класифікації вікових груп: F1-метрики та візуалізація матриці невідповідностей**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

age_bins_labels = [
    '1–9', '10–20', '21–26', '27–34',
    '35–45', '45–60', '60–99'
]

Y_pred = model_mobilenet.predict(X_test)
Y_pred_classes = np.argmax(Y_pred, axis=1)
Y_true = np.argmax(Y_test_encoded, axis=1)

from sklearn.metrics import classification_report, f1_score

# Обчислення F1-score
f1_macro = f1_score(Y_true, Y_pred_classes, average='macro')
f1_weighted = f1_score(Y_true, Y_pred_classes, average='weighted')

print(f"F1-score (macro): {f1_macro:.4f}")
print(f"F1-score (weighted): {f1_weighted:.4f}")

conf_matrix = confusion_matrix(Y_true, Y_pred_classes)

conf_matrix_percent = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis] * 100

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_percent, annot=True, fmt='.1f', cmap='Blues', 
            xticklabels=age_bins_labels, yticklabels=age_bins_labels)

plt.xlabel('Прогнозовані мітки')
plt.ylabel('Істинні мітки')
plt.title('Матриця невідповідностей (%)')
plt.show()

**Візуалізація метрик навчання моделі MobileNet для задачі класифікації із застосуванням балансування класів**

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import load_model
import pickle
model_mobilenet_balanced = load_model('/kaggle/input/mobilenet-class-56/mobilenet_clas_regression (3).h5')

with open('/kaggle/input/mobilenet-class-56/mobilenet_model_clas_history (1).pkl', 'rb') as f:
    history_mobilenet_clas_balanced = pickle.load(f)
    
train_accuracy = history_mobilenet_clas_balanced['accuracy']
val_accuracy = history_mobilenet_clas_balanced['val_accuracy']
train_loss = history_mobilenet_clas_balanced['loss']
val_loss = history_mobilenet_clas_balanced['val_loss']

plt.figure(figsize=(10, 10))

plt.subplot(2, 1, 1)
plt.plot(train_accuracy, label='Тренувальна точність')
plt.plot(val_accuracy, label='Валідаційна точність')
plt.title('Точність моделі')
plt.xlabel('Епохи')
plt.ylabel('Точність')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Втрати моделі')
plt.xlabel('Епохи')
plt.ylabel('Втрати')
plt.legend()

plt.tight_layout()
plt.show()

**Оцінка якості класифікації вікових груп після балансування: F1-метрики та візуалізація матриці невідповідностей**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

age_bins_labels = [
    '1–9', '10–20', '21–26', '27–34',
    '35–45', '45–60', '60–99'
]

Y_pred = model_mobilenet_balanced.predict(X_test)
Y_pred_classes = np.argmax(Y_pred, axis=1)
Y_true = np.argmax(Y_test_encoded, axis=1)

from sklearn.metrics import classification_report, f1_score

# Обчислення F1-score
f1_macro = f1_score(Y_true, Y_pred_classes, average='macro')
f1_weighted = f1_score(Y_true, Y_pred_classes, average='weighted')

print(f"F1-score (macro): {f1_macro:.4f}")
print(f"F1-score (weighted): {f1_weighted:.4f}")

conf_matrix = confusion_matrix(Y_true, Y_pred_classes)

conf_matrix_percent = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis] * 100

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_percent, annot=True, fmt='.1f', cmap='Blues', xticklabels=age_bins_labels, 
            yticklabels=age_bins_labels)

plt.xlabel('Прогнозовані мітки')
plt.ylabel('Істинні мітки')
plt.title('Матриця невідповідностей (%)')
plt.show()
